In [14]:
import spacy
from sklearn import preprocessing
from sklearn_crfsuite import CRF
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split

import pycrfsuite as crfs

import json
import os
import re
from time import time
from nltk.tokenize import sent_tokenize


In [8]:
# Hacer funciron que lea las sentencias del documento de sentencias y las pase por la siguiente funcion iterativamente (solo las que tengan negación. Así que coger el fichero de negaciones y mirar si hay una en esa frase y entonce pasar)
# Hacer una funcion que convierta cada sentencia en una lista de tokens con las features correspondientes
# Pasar lo que devuelva la funcion a una lista mas grando, qie serà el training dataset
# Hacer  una función para passar las sentencias al CRF y que se entrene (basado en el scope)
# Hacer una función para que testear el funcionamiento del CRF

In [53]:
# Load the Spanish language model
nlp = spacy.load("es_core_news_sm")

# Function to read sentences from a file and store them in a dictionary
def read_file(file_path):
    sentences = []
    with open(file_path, 'r') as file:
        for line in file:
            sentences.append(line.strip())
    return sentences

# Function to process text data
def process_text(data, negations, output_file):
    sentences=[]
    # Iterate over each data item
    #!for i in range(len(data)): LO HAGO SOLO CON EL PROMERO PARA PROBAR SI FUNCIONA Y PARA QUE VAYA RAPIDO
    text = data[0]["data"]["text"]
    sentences.extend(sent_tokenize(text))
    return sentences

def get_only_negated_sentences(sentences, negations):
    negated_sentences=[]
    for sentence in sentences:
        for negation in negations:
            pattern = r'\b' + re.escape(negation) + r'\b'
            if re.search(pattern, sentence):
                negated_sentences.append(sentence)
                break
    return negated_sentences

def from_sentences_to_tokenfeatures(sentences):
    feature_sentences=[]
    # Process each sentence
    for sentence in sentences:
        feature_words=[]
        # Process the sentence with SpaCy
        doc = nlp(sentence)
        
        # Extract base forms, part-of-speech tags, chunk tags, and named entity tags
        base_forms = [token.lemma_ for token in doc]
        pos_tags = [token.pos_ for token in doc]
        chunk_tags = [token.dep_ for token in doc]
        
        # Output the results in a tab-separated format
        for token, lemma, pos, chunk in zip(doc, base_forms, pos_tags, chunk_tags):
            #print(f"{token.text}\t{lemma}\t{pos}\t{chunk}\t{ner}")
            feature_words.append([token.text, lemma, pos, chunk])
        #print()  # Separate sentences with an empty line
        feature_sentences.append(feature_words)

    return feature_sentences

def find_sublist(lst, sublist):
    for i in range(len(lst) - len(sublist) + 1):
        if lst[i:i+len(sublist)] == sublist:
            return True, i
    return False, -1

def label_sublist(words_list, sublist, label, labeled_list=[]):
    if(not labeled_list):
        labeled_list = ['O'] * len(words_list)
    found, start_index = find_sublist(words_list, sublist)
    if not found:
        return None

    for i, word in enumerate(words_list):
        if i == start_index:
            labeled_list[i] = 'B-'+label
        elif start_index < i < start_index + len(sublist):
            labeled_list[i] = 'I-'+label

    return labeled_list

def from_sentences_to_BIO_tagging(sentences, words, label): #?Si añado labeled_words=[] parece que guarga lo del training data. Buscar una forma de añadir la lista. Puede que con un booleano extra para determinar si se borra la lista o no
    #? ESTO HACERLO EN OTRA FUNCIÓN, PORQUE ES LA Y, Y LAS FEATURES SON LAS X, ADEMAS DE QUE LAS FEATUES SE REPETIRIAN CAVEZ QUE LLAMAMOS A LA FUNCIÓN PARA CALCULAR LOS BOI TAGS CORRESPONDIENTES
    #? HACER UNA LISTA EN VEZ DE UN DICCIONARIO
    #? BUSCAR COMO HACER QUE LA LISTA SE VAYA ACTUALIZANDO CADA VEZ QUE SE ENCUENTRA UN NUEVO BIO TAG, PARA QUE NO DESAPAREZCA EL ANTERIOR Y NO SEAN LISTAS DIFERENTES
    #Extract BIO tagging for negations, uncertanties and its scopes
    labeled_words=[]
    for sentence in sentences:
        doc = nlp(sentence)
        sentence_list=[token.text for token in doc]
        labeled_list=[]
        for i, word in enumerate(words):
            word_list=word.split()
            
            possible_labeled_list = label_sublist(sentence_list, word_list, label, labeled_list)
            if possible_labeled_list:
                labeled_list=possible_labeled_list
                #print(str(sentence_list)+" "+str(len(sentence_list)))
                #print(str(labeled_list)+" "+str(len(labeled_list)))
        #print(str(labeled_list)+" "+str(len(labeled_list)))
        labeled_words.append(labeled_list)
    for i in labeled_words:
        print(i)
    return labeled_words

# Define feature extraction function
def token2features(tokens, i):
    token = tokens[i]
    features = {
        'token': token[0],        # The token itself
        'lemma': token[1],        # Lemma
        'pos_tag': token[2],      # POS tagging
        'chunk': token[3],        # Chunk
        'bias': 1.0
    }
    if i > 0:
        prev_token = tokens[i-1]
        features.update({
            'prev_token': prev_token[0],
            'prev_pos_tag': prev_token[2],
            'prev_chunk': prev_token[3]
        })
    else:
        features['BOS'] = True  # Beginning of sequence
    if i < len(tokens)-1:
        next_token = tokens[i+1]
        features.update({
            'next_token': next_token[0],
            'next_pos_tag': next_token[2],
            'next_chunk': next_token[3]
        })
    else:
        features['EOS'] = True  # End of sequence
    return features


ROOT_DIR = os.path.dirname(os.path.abspath(""))
negations_file = '../data/negation_cue.txt'
output_file_training = "../data/output_blind_negex_training.json"
output_file_testing = "../data/output_blind_negex_testing.json"

# Load training data and process
data_training = json.load(open(os.path.join(ROOT_DIR, "data", "training_data.json")))
data_testing = json.load(open(os.path.join(ROOT_DIR, "data", "test_data.json")))
negations = read_file(negations_file)

sentences_train=process_text(data_training, negations, output_file_training)
print(len(sentences_train))
negated_sentences_train=get_only_negated_sentences(sentences_train, negations)
print(len(negated_sentences_train))
X_training=from_sentences_to_tokenfeatures(negated_sentences_train)
Y_training=from_sentences_to_BIO_tagging(negated_sentences_train, negations, "NEG")
"""y_train=from_sentences_to_BIO_tagging(negated_sentences, negations, "NEG", y_train)
y_train=from_sentences_to_BIO_tagging(negated_sentences, negations, "NEG", y_train)
y_train=from_sentences_to_BIO_tagging(negated_sentences, negations, "NEG", y_train)"""
#TODO Me faltaria decir cual es la negación y el scope, con el BIO tagging. También pensar en como entrenar al CRF para que sepa que solo tiene que poner el scope y lo otro como O 
#TODO Continuación: podría decir que haga el BIO tagging de toda la frase, de tal modo que tambien diria el BIO tag de las negaciones. Claro, y el modelo entonces tendria que encontrar las negaciones y el scope, todo junto
#TODO:
# Pasar lo que devuelva la funcion a una lista mas grando, qie serà el training dataset
# Hacer  una función para passar las sentencias al CRF y que se entrene (basado en el scope)
# Hacer una función para que testear el funcionamiento del CRF

# Hacer un txt para negations, otro para uncertanty, otro para scopes negation y otro para scopes uncertanty

sentences_test=process_text(data_testing, negations, output_file_testing)
print(len(sentences_test))
negated_sentences_test=get_only_negated_sentences(sentences_test, negations)
print(len(negated_sentences_test))
X_testing=from_sentences_to_tokenfeatures(negated_sentences_test)
Y_testing=from_sentences_to_BIO_tagging(negated_sentences_test, negations, "NEG")

"""for xseq, yseq in zip(X_training, Y_training):
    print(xseq, yseq)"""

28
11
['B-NEG', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-

'for xseq, yseq in zip(X_training, Y_training):\n    print(xseq, yseq)'

In [54]:
# Prepare data
X_train = []
y_train = []

for sentence_tokens, sentence_labels in zip(X_training, Y_training):
    X_sentence = [token2features(sentence_tokens, i) for i in range(len(sentence_tokens))]
    y_sentence = sentence_labels
    X_train.append(X_sentence)
    y_train.append(y_sentence)


# Train the CRF model
trainer = crfs.Trainer(verbose=False)
for xseq, yseq in zip(X_train, y_train):
    print(len(xseq), yseq)
    trainer.append(xseq, yseq)
trainer.set_params({'c1': 1.0, 'c2': 1e-3, 'max_iterations': 50, 'feature.possible_transitions': True})
trainer.train('../data/bio_crf.crfsuite')

# Make predictions
tagger = crfs.Tagger()
tagger.open('../data/bio_crf.crfsuite')

X_test = []
y_test = []

for sentence_tokens, sentence_labels in zip(X_testing, Y_testing):
    X_sentence = [token2features(sentence_tokens, i) for i in range(len(sentence_tokens))]
    y_sentence = sentence_labels
    print(y_sentence)
    X_test.append(X_sentence)
    y_test.append(y_sentence)

y_pred = [tagger.tag(xseq) for xseq in X_test]



4 ['B-NEG', 'O', 'O', 'O']
52 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
12 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O']
33 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O']
26 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
28 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
47 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [55]:
for xseq, yseq in zip(X_test, y_test):
    print(len(xseq), yseq)

142 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
3 ['O', 'B-NEG', 'O']
18 ['O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O']
39 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NEG', 'O', 'O', 'O', 'O', 

In [56]:
for sentence, y_p, y_t in zip(X_test, y_pred, y_test):
    print(len(sentence), y_p, y_t)

142 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',